In [1]:
# 메모리 사용량 측정
import torch

def print_gpu_utilization():
    if torch.cuda.is_available():
        used_memory = torch.cuda.memory_allocated() / 1024**3
        print(f"GPU memory used: {used_memory:.3f} GB")
    else:
        print("GPU not available")

print_gpu_utilization()

GPU not available


In [ ]:
# 모델을 불러오고 GPU 메모리와 데이터 타입 확인

from transformers import AutoModelForCausalLM, AutoTokenizer

def load_model_and_tokenizer(model_id, peft=None):
    tokenizer = AutoTokenizer.from_pretrained(model_id)
    if peft is None:
        model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype="auto", device_map={"":0})

    print_gpu_utilization()

    return model, tokenizer

model_id = "EleutherAI/polyglot-ko-1.3b"
model, tokenizer = load_model_and_tokenizer(model_id)   # GPU 메모리 사용량 : 2.599 GB
print("model parameter data type: ", model.dtype)   # torch.float16

In [ ]:
# gradient 와 optimizer state 의 메모리 사용량을 계산하는 함수

from transformers import AdamW
from torch.utils.data import DataLoader

def estimate_memory_of_gradients(model):
    total_memory = 0
    for param in model.parameters():
        if param.grad is not None:
            total_memory += param.grad.nelement() * param.grad.element_size()   # 모델에 저장된 gradient 개수 * gradient 데이터 크기
    return total_memory

def estimate_memory_of_optimizer(optimizer):
    total_memory = 0
    for state in optimizer.state.values():
        for k, v in state.items():
            if torch.is_tensor(v):
                total_memory += v.nelement() * v.element_size()
    return total_memory

In [ ]:
# 모델의 학습 과정에서 메모리 사용량을 확인하는 train_model 정의

def train_model(model, dataset, training_args):
    if training_args.gradient_checkpointing:
        # 순전파 중 특정 지점(checkpoint)의 활성화값만 저장하여 메모리 사용량을 줄임.
        # 다만, 역전파 시 checkpoint에 포함되지 않은 중간 활성화값을 다시 계산해야 하므로
        # 전체 연산량이 증가함.
        model.gradient_checkpointing_enable()

    train_dataloader = DataLoader(dataset, batch_size=training_args.per_device_train_batch_size)
    optimizer = AdamW(model.parameters())
    model.train()

    gpu_utilization_printed = False

    for step, batch in enumerate(train_dataloader, start=1):
        batch = {k: v.to(model.device) for k, v in batch.items()}

        outputs = model(**batch)
        loss = outputs.loss
        loss = loss / training_args.gradient_accumulation_steps
        # gradient_accumulation_steps:
        #   gradient_accumulation_steps만큼 backward를 수행해서
        #   한 번의 optimizer.step()을 할지 정하는 값입니다.
        #   (여러 번의 누적 gradient로, 큰 배치 효과를 얻으면서 메모리 부담을 줄임)
        loss.backward()

        if step % training_args.gradient_accumulation_steps == 0:
            optimizer.step()    # 누적된 gradient로 모델 파라미터 업데이트
            gradients_memory = estimate_memory_of_gradients(model)
            optimizer_memory = estimate_memory_of_optimizer(optimizer)

            if not gpu_utilization_printed:
                print_gpu_utilization()
                gpu_utilization_printed = True

            optimizer.zero_grad()

    print(f"optimizer state memory: {optimizer_memory/1024**3:.3f} GB")
    print(f"gradients memory: {gradients_memory/1024**3:.3f} GB")

In [ ]:
# 랜덤 데이터셋을 생성하는 make_dummy_dataset 정의

import numpy as np
from datasets import Dataset

def make_dummy_dataset():
    seq_len, dataset_size = 256, 64
    dummy_data = {
        "input_ids": np.random.randint(100, 30000, (dataset_size, seq_len)),
        "labels": np.random.randint(100, 30000, (dataset_size, seq_len))
    }
    dataset = Dataset.from_dict(dummy_data)
    dataset.set_format("pt")    # 데이터셋을 tf.Tensor 로 변환
    return dataset

In [ ]:
# 더 이상 사용하지 않는 GPU 메모리를 반환하는 cleanup 함수

import gc

def cleanup():
    if 'model' in globals():
        del globals()['model']

    if 'dataset' in globals():
        del globals()['dataset']

    gc.collect()    # 사용하지 않는 메모리를 회수하는 가비지 컬렉션(garbage collection) 수행
    torch.cuda.empty_cache()    # 더 이상 사용하지 않는 GPU 메모리 반환

In [ ]:
# GPU 사용량을 확인하는 gpu_memory_experiment 함수 정의

from transformers import TrainingArguments, Trainer

def gpu_memory_experiment(batch_size,
                          gradient_accumulation_steps=1,
                          gradient_checkpointing=False,
                          model_id="EleutherAI/polyglot-ko-1.3b",
                          peft=None):

    print(f"Batch size: {batch_size}")

    model, tokenizer = load_model_and_tokenizer(model_id, peft=peft)

    if gradient_checkpointing==True or peft == "qlora":
        model.config.use_cache = False

    dataset = make_dummy_dataset()

    training_args = TrainingArguments(
        per_device_train_batch_size=batch_size,
        gradient_accumulation_steps=gradient_accumulation_steps,
        gradient_checkpointing=gradient_checkpointing,
        output_dir="./result",
        num_train_epochs=1
    )

    try:
        train_model(model, dataset, training_args)
    except RuntimeError as e:
        if "CUDA out of memory" in str(e):
            print(e)
        else:
            raise e
    finally:
        del model, dataset
        gc.collect()
        torch.cuda.empty_cache()
        print_gpu_utilization()

In [ ]:
# 배치 크기를 변경하며 메모리 사용량 측정
cleanup()
print_gpu_utilization()

for batch_size in [4, 8, 16]:
    gpu_memory_experiment(batch_size)

    torch.cuda.empty_cache()